<a href="https://colab.research.google.com/github/mantuonweb/Google_Collab/blob/master/Agent_Resume_Matcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install langchain-openai langchain-community langchain-text-splitters langchain-core faiss-cpu python-dotenv pypdf

In [ ]:
import os
import shutil
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader, PyPDFLoader
from dotenv import load_dotenv
from google.colab import userdata

# Load API key
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
load_dotenv()
print("✓ Setup complete")
print("API Key:", os.environ.get("OPENAI_API_KEY", "Not set")[:20] + "...")

# Create folders if they don't exist
os.makedirs("./resumes", exist_ok=True)
os.makedirs("./resume_db", exist_ok=True)
print("✓ Folders created: ./resumes and ./resume_db")


✓ Setup complete
API Key: sk-proj-2D_k1B8OV3MW...
✓ Folders created: ./resumes and ./resume_db


In [ ]:
import shutil
import os

# Remove corrupted database
if os.path.exists("./resume_db"):
    shutil.rmtree("./resume_db")
    print("✓ Cleaned up old database")

# Recreate folder
os.makedirs("./resume_db", exist_ok=True)
print("✓ Ready for fresh start")


✓ Cleaned up old database
✓ Ready for fresh start


In [ ]:
import shutil
import os
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader, PyPDFLoader
from dotenv import load_dotenv
from google.colab import userdata

# Initialize embeddings globally
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

def ingest_resumes():
    """Load resumes from ./resumes folder and add to vector database"""
    print("📥 Ingesting resumes...")

    # Load text files
    txt_loader = DirectoryLoader("./resumes", glob="**/*.txt", loader_cls=TextLoader)
    txt_docs = txt_loader.load()

    # Load PDF files
    pdf_loader = DirectoryLoader("./resumes", glob="**/*.pdf", loader_cls=PyPDFLoader)
    pdf_docs = pdf_loader.load()

    all_docs = txt_docs + pdf_docs

    if not all_docs:
        print("❌ No resumes found in ./resumes folder")
        return

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(all_docs)

    # Check if FAISS index file exists (not just folder)
    db_file_exists = os.path.exists("./resume_db/index.faiss")

    if db_file_exists:
        # Load existing and add new documents
        vectorstore = FAISS.load_local("./resume_db", embeddings, allow_dangerous_deserialization=True)
        vectorstore.add_documents(chunks)
        print(f"✓ Added {len(chunks)} chunks from {len(all_docs)} resumes")
    else:
        # Create new vector store
        vectorstore = FAISS.from_documents(chunks, embeddings)
        print(f"✓ Created new database with {len(chunks)} chunks from {len(all_docs)} resumes")

    vectorstore.save_local("./resume_db")
    print("✓ Database saved successfully")


def list_resumes():
    """List all resumes stored in vector database"""
    print("📋 Listing resumes...")

    if not os.path.exists("./resume_db/index.faiss"):
        print("❌ No database found. Please ingest resumes first.")
        return

    vectorstore = FAISS.load_local("./resume_db", embeddings, allow_dangerous_deserialization=True)

    # Get all documents
    all_docs = vectorstore.docstore._dict

    # Extract unique sources
    sources = set()
    for doc in all_docs.values():
        if hasattr(doc, 'metadata') and 'source' in doc.metadata:
            sources.add(os.path.basename(doc.metadata['source']))

    print(f"\n✓ Found {len(sources)} resumes in database:")
    for i, source in enumerate(sorted(sources), 1):
        print(f"  {i}. {source}")


def search_resumes(skills):
    """Search resumes by skills and return best matches"""
    print(f"🔍 Searching for candidates with skills: {skills}")

    if not os.path.exists("./resume_db/index.faiss"):
        print("❌ No database found. Please ingest resumes first.")
        return

    vectorstore = FAISS.load_local("./resume_db", embeddings, allow_dangerous_deserialization=True)

    # Search for relevant resume chunks
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
    docs = retriever.invoke(skills)

    # Create context from retrieved documents
    context = "\n\n".join([f"Resume {i+1}:\n{doc.page_content}" for i, doc in enumerate(docs)])

    # Create prompt for LLM
    prompt = f"""You are a recruiter assistant. Based on the following resume excerpts, identify and rank the best candidates for the required skills.\n\nRequired Skills: {skills}\n\nResume Excerpts:\n{context}\n\nPlease provide a quick summary for the top 3 best matching candidates. For each candidate, include their relevant skills, why they are a good fit, and a matching percentage. The response should be concise.\n\nAnswer:"""

    # Get LLM response
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    response = llm.invoke(prompt)

    print("\n" + "="*60)
    print("🎯 SEARCH RESULTS")
    print("="*60)
    print(response.content)
    print("="*60)

    return response.content


def clear_resumes():
    """Clear all resumes from vector database"""
    print("🗑️  Clearing resume database...")

    if os.path.exists("./resume_db"):
        shutil.rmtree("./resume_db")
        print("✓ Database cleared successfully")
    else:
        print("❌ No database found")

print("✓ Agent functions loaded successfully")

✓ Agent functions loaded successfully


In [ ]:
def generate_resume(data):
    """Generate a text resume from data dictionary"""
    resume = []

    # Header
    resume.append(data['name'].upper())
    resume.append(f"{data['email']} | {data['phone']} | {data['location']}")
    resume.append("")

    # Skills
    resume.append("SKILLS")
    resume.append(", ".join(data['skills']))
    resume.append("")

    # Experience
    resume.append("EXPERIENCE")
    for exp in data['experiences']:
        resume.append(f"{exp['title']} | {exp['company']} | {exp['duration']}")
        for resp in exp['responsibilities']:
            resume.append(f"- {resp}")
        resume.append("")

    # Education
    resume.append("EDUCATION")
    edu = data['education']
    resume.append(f"{edu['degree']} | {edu['institution']} | {edu['year']}")

    return "\n".join(resume)


def save_resume(data, filepath):
    """Save resume to file"""
    with open(filepath, 'w') as f:
        f.write(generate_resume(data))


# Example usage
if __name__ == "__main__":
    resume_data = {
        "name": "Mantu Nigam",
        "email": "mantu.nigam@email.com",
        "phone": "+91-9876543210",
        "location": "Bangalore",
        "skills": ["Python", "LangChain", "OpenAI", "RAG", "FAISS", "FastAPI", "AWS", "Docker"],
        "experiences": [
            {
                "title": "Senior AI Engineer",
                "company": "TechCorp",
                "duration": "2021-Present",
                "responsibilities": [
                    "Built RAG applications with LangChain and OpenAI",
                    "Developed chatbots using FAISS vector database"
                ]
            },
            {
                "title": "Software Engineer",
                "company": "DataMinds",
                "duration": "2019-2021",
                "responsibilities": [
                    "Created ML models and REST APIs with Python"
                ]
            }
        ],
        "education": {
            "degree": "B.Tech CS",
            "institution": "IIT Delhi",
            "year": "2019"
        }
    }

    # Generate and print
    print(generate_resume(resume_data))

    # Save to file
    save_resume(resume_data, "resumes/mantu_nigam_resume.txt")


MANTU NIGAM
mantu.nigam@email.com | +91-9876543210 | Bangalore

SKILLS
Python, LangChain, OpenAI, RAG, FAISS, FastAPI, AWS, Docker

EXPERIENCE
Senior AI Engineer | TechCorp | 2021-Present
- Built RAG applications with LangChain and OpenAI
- Developed chatbots using FAISS vector database

Software Engineer | DataMinds | 2019-2021
- Created ML models and REST APIs with Python

EDUCATION
B.Tech CS | IIT Delhi | 2019


In [ ]:
# Add resumes to database
ingest_resumes()


📥 Ingesting resumes...
✓ Created new database with 2 chunks from 2 resumes
✓ Database saved successfully


In [ ]:
# Search for candidates with specific skills
skills = "front end, angular, react, microservice using nestjs"  # Change this to your required skills
search_resumes(skills)

🔍 Searching for candidates with skills: front end, angular, react, microservice using nestjs

🎯 SEARCH RESULTS
### Top Candidates Summary

**1. Vinod Malik**  
- **Relevant Skills:** React, Node.js (Full Stack Development)  
- **Why They Are a Good Fit:** Vinod has hands-on experience with React, which is a key requirement. His role as a Full Stack Developer indicates familiarity with front-end technologies and the ability to work on web applications. However, he lacks direct experience with Angular and microservices using NestJS.  
- **Matching Percentage:** 70%

---

**2. Mantu Nigam**  
- **Relevant Skills:** None directly related to front-end, Angular, React, or microservices using NestJS  
- **Why They Are a Good Fit:** Mantu has strong technical skills in AI and backend development but does not possess any relevant experience in front-end technologies or the specific frameworks required.  
- **Matching Percentage:** 10%

---

**3. (No third candidate)**  
- **Relevant Skills:** N

'### Top Candidates Summary\n\n**1. Vinod Malik**  \n- **Relevant Skills:** React, Node.js (Full Stack Development)  \n- **Why They Are a Good Fit:** Vinod has hands-on experience with React, which is a key requirement. His role as a Full Stack Developer indicates familiarity with front-end technologies and the ability to work on web applications. However, he lacks direct experience with Angular and microservices using NestJS.  \n- **Matching Percentage:** 70%\n\n---\n\n**2. Mantu Nigam**  \n- **Relevant Skills:** None directly related to front-end, Angular, React, or microservices using NestJS  \n- **Why They Are a Good Fit:** Mantu has strong technical skills in AI and backend development but does not possess any relevant experience in front-end technologies or the specific frameworks required.  \n- **Matching Percentage:** 10%\n\n---\n\n**3. (No third candidate)**  \n- **Relevant Skills:** N/A  \n- **Why They Are a Good Fit:** There are no additional candidates provided in the excer

In [ ]:
# Simple interactive menu
def run_agent():
    while True:
        print("\n" + "="*60)
        print("📄 RESUME AGENT - MENU")
        print("="*60)
        print("1. Ingest Resumes")
        print("2. List Resumes")
        print("3. Search by Skills")
        print("4. Clear Database")
        print("5. Exit")
        print("="*60)

        choice = input("\nEnter your choice (1-5): ")

        if choice == "1":
            ingest_resumes()
        elif choice == "2":
            list_resumes()
        elif choice == "3":
            skills = input("Enter required skills (comma-separated): ")
            search_resumes(skills)
        elif choice == "4":
            confirm = input("Are you sure? (yes/no): ")
            if confirm.lower() == "yes":
                clear_resumes()
        elif choice == "5":
            print("👋 Goodbye!")
            break
        else:
            print("❌ Invalid choice. Please try again.")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')